In [ ]:
## Load the LabOne API and other necessary packages
from zhinst.toolkit import Session,  Waveforms
from zhinst.toolkit import CommandTable
import numpy as np

device_id='DEV9068'
server_host = 'localhost'

### connect to data server
session = Session(server_host)
### connect to device
device = session.connect_device(device_id)


# sequence parameters
nu = 244.1#385 #385 #100 #100#388#385.7 # microwave frequency in MHz
Fs = 2400 
NUM_SUBCYCLES = 150  # Number of subcycles #Number of points
tau_step0 =1.25E-3 # tau step in us
tau_start0 = 2.5E-3 # starting tau in us
t_tot = 8.76 # total time of a pulse frame in us 
TT_tot = t_tot*NUM_SUBCYCLES*2  # total time need for measuring all data points once
t_initi = 5 #laser initialization time
A = 0.9 #0.65#0.5087 #0.6 #0.3393#0.3#0.7
relative_phase = 0



#A = 0.36 # 100 T=35ns

# convert parameters
TAU_STEP = int(tau_step0*Fs) # convert tau step into tick
#TAU_START = int(tau_start0*Fs) # convert tau step into tick
DATA_POINTS_PER_SUBCYCLE = int(t_tot*Fs)  # Data points per subcycle
DATA_POINTS_TOTAL = DATA_POINTS_PER_SUBCYCLE*NUM_SUBCYCLES*2 # total waveform points for measuring all data points once
INITIAL_PERIOD_LENGTH =  int(tau_start0*Fs)  # Initial tau
MAX_PERIOD_LENGTH = INITIAL_PERIOD_LENGTH + (NUM_SUBCYCLES - 1) * TAU_STEP  # Maximum length of tau
WFM_LEN = NUM_SUBCYCLES * DATA_POINTS_PER_SUBCYCLE*2  # Total waveform length
T_INITIAL = int((t_initi+0.12)*Fs)



# sequence parameters
# nu14 = 150 # microwave frequency in MHz, drive III-2
# nu23 = -150 # microwave frequency in MHz, drive III-4
# RF_freq =  147.4 # RF frequency in MHz, drive nuclear spin 
# nu_RF = 20 # generated RF field to be sensed
# Fs = 2400 # Sampling rate in MHz
# NUM_SUBCYCLES = 100  # Number of subcycles
# tauCPMG = 6.5E-3

# tau_start0 = 0000E-3 # starting tau in us
# tau_step0 = 5E-3# tau step in us (nuclear memory time)

# t_initi = 6; #laser initialization time
# t_wait = 3 # waiting time between first and second laser in us
# t_sensing = 0.5 
# t_store = 1
# t_sensing2 = 1.5
# t_read = 5
# t_tot = t_initi+t_wait+ t_initi+t_sensing+ t_store +t_initi+t_sensing2+t_read# total time of a pulse frame in us. exclude memory time
# t_RF = 2.5;
# print(t_tot)
# rabi_freq_esingle = 39.92 # non-local rabi frequency
# rabi_freq_e = 26.46 # local  rabi frequency in MHz
# rabi_freq_N = 1 # nuclear spin rabi frequency in MHz

# TT_tot = t_tot*NUM_SUBCYCLES*2+ (tau_start0*NUM_SUBCYCLES+tau_step0*NUM_SUBCYCLES*(NUM_SUBCYCLES-1)/2)*2 ; # total time need for measuring all data points once

# N = 8 # pi_y repeat times
# A14 = 0.5
# A23 = 0.5
# A_n=0.4 #amplitude of nuclear spin driving
# A_RF =1*3/4

# # convert parameters
# TAU_STEP = int(tau_step0*Fs) # convert tau step into tick
# TAU_START = int(tau_start0*Fs) # convert tau step into tick
# DATA_POINTS_PER_SUBCYCLE = int(t_tot*Fs)  # Data points per subcycle

# INITIAL_PERIOD_LENGTH =  int(tau_start0*Fs)  # Initial tau
# MAX_PERIOD_LENGTH = INITIAL_PERIOD_LENGTH + (NUM_SUBCYCLES - 1) * TAU_STEP  # Maximum length of tau
# WFM_LEN = int(TT_tot*Fs) # Total waveform length
# DATA_POINTS_TOTAL = WFM_LEN # total waveform points for measuring all data points once

# ONE_PI_E = int(Fs/(2*rabi_freq_esingle))  # non-local pi pulse points, electron
# ONE_PI_E_LOCAL = int(Fs/(2*rabi_freq_e)) # local pi pulse points, electron
# HALF_PI_E = int(Fs/(4*rabi_freq_esingle))  # non-local pi pulse points, electron
# HALF_PI_E_LOCAL = int(Fs/(4*rabi_freq_e)) # local pi pulse points, electron

# ONE_PI_N = int(Fs/(2*rabi_freq_N)) 
# Length_firstLoop =  DATA_POINTS_PER_SUBCYCLE + TAU_START

# T_MW_RF_separation = int((0.125)*Fs)
# T_WAITING = int((t_wait)*Fs)
# T_SENSING = int((t_sensing)*Fs)
# T_SOTRE = int((t_store)*Fs)
# T_SENSING2 = int((t_sensing2)*Fs)
# T_INITIAL = int((t_initi)*Fs)
# T_IDLE = int((0.12)*Fs)
# T_RF = int(t_RF*Fs)


# define sequence veriable and send it the sequencer in LabOne

seqc_a = """\
const LENGTH = """
seqc_b = str(DATA_POINTS_TOTAL)
seqc_c = """; // Number of subcycles * Data points per subcycle

wave w1 = placeholder(LENGTH, false, false); // Create a waveform of size LENGTH, with one marker
wave w2 = placeholder(LENGTH, false, false); // Create a waveform of size LENGTH, with one marker

assignWaveIndex(w1,w2,10);                 // Create a wave table entry with placeholder waveform              
                                         // routed to output 1, with index 10

// execute the command table entry
executeTableEntry(0);

while(true){
  waitDigTrigger(1);
  playWave(w1,w2);

 waitWave();
};
"""
seqc_c2 = """; // Number of subcycles * Data points per subcycle

wave w3 = placeholder(LENGTH, false, false); // Create a waveform of size LENGTH, with one marker
wave w4 = placeholder(LENGTH, false, false); // Create a waveform of size LENGTH, with one marker


assignWaveIndex(w3,w4,11);                 // Create a wave table entry with placeholder waveform              
                                         // routed to output 3, with index 10
// execute the command table entry
executeTableEntry(0);

while(true){
  waitDigTrigger(1);
  playWave(w3,w4);
  waitWave();
};
"""

seqc_program = seqc_a + seqc_b +seqc_c
seqc_program2 = seqc_a + seqc_b +seqc_c2
print(seqc_program)
print(seqc_program2)

AWG_INDEX1 = 0 #use sequencer 1, channel 1&2 for 2*2
AWG_INDEX2 = 1 #use sequencer 2, channel 3&4 for 2*2
awg = device.awgs[AWG_INDEX1]
awg2 = device.awgs[AWG_INDEX2]
WAVE_INDEX = 0
TABLE_INDEX = 0

# load the sequence
awg.load_sequencer_program(seqc_program)
awg2.load_sequencer_program(seqc_program2)


wave1 = np.zeros(WFM_LEN)
wave2 = np.zeros(WFM_LEN)
wave3 = np.zeros(WFM_LEN)
wave4 = np.zeros(WFM_LEN)



for subcycle in range(NUM_SUBCYCLES):
    current_length = INITIAL_PERIOD_LENGTH + subcycle * TAU_STEP  # Length for the current sine wave
    start_index = subcycle * DATA_POINTS_PER_SUBCYCLE*2 + T_INITIAL # Center position
    
    # Generate sine wave
    for i in range(current_length):
        wave1[start_index + i] = A*np.sin( 2*np.pi*i*nu/Fs)  # Sine wave calculation

# Generate cosine waves for each subcycle
for subcycle in range(NUM_SUBCYCLES):
    current_length = INITIAL_PERIOD_LENGTH + subcycle * TAU_STEP  # Length for the current sine wave
    start_index = subcycle * DATA_POINTS_PER_SUBCYCLE*2 + T_INITIAL# Center position
    
    # Generate sine wave
    for i in range(current_length):
        wave2[start_index + i] = A*np.cos( 2*np.pi*i*nu/Fs)  # Sine wave calculation


for subcycle in range(NUM_SUBCYCLES):
    current_length = INITIAL_PERIOD_LENGTH + subcycle * TAU_STEP  # Length for the current sine wave
    start_index = subcycle * DATA_POINTS_PER_SUBCYCLE*2 + T_INITIAL # Center position
    
    # Generate sine wave
    for i in range(current_length):
        wave3[start_index + i] = A*np.sin( 2*np.pi*i*nu + relative_phase)  # Sine wave calculation

# Generate cosine waves for each subcycle
for subcycle in range(NUM_SUBCYCLES):
    current_length = INITIAL_PERIOD_LENGTH + subcycle * TAU_STEP  # Length for the current sine wave
    start_index = subcycle * DATA_POINTS_PER_SUBCYCLE*2 + T_INITIAL# Center position
    
    # Generate sine wave
    for i in range(current_length):
        wave4[start_index + i] = A*np.cos( 2*np.pi*i*nu + relative_phase)  # Sine wave calculation



# Generate sine waves for each subcycle

# for subcycle in range(NUM_SUBCYCLES):
#     #current_length = INITIAL_PERIOD_LENGTH + subcycle * TAU_STEP  # Length for the current sine wave
#     current_length = int(tauCPMG*Fs)
#     start_index = subcycle * DATA_POINTS_PER_SUBCYCLE*2+ int(TAU_START*subcycle+TAU_STEP*subcycle*(subcycle-1)/2)*2 + T_INITIAL + T_IDLE # starting position
#     # intialize nuclear spin
#     for i in range(ONE_PI_E + 1):
#         wave1[start_index + i] = A14*np.sin(2*np.pi*i*nu14/Fs) #nonlocal gate to flip the electron spin
#         wave2[start_index + i] = A14*np.cos(2*np.pi*i*nu14/Fs) #nonlocal gate to flip the electron spin
    
#     for i in range(ONE_PI_N + 1):
#         wave3[start_index + i + ONE_PI_E+ T_MW_RF_separation] = A_n*np.sin(2*np.pi*i*RF_freq/Fs)
    
#     # sensing with CPMG using local gate
#     start_index = start_index + T_WAITING + T_INITIAL
#     for i in range(T_RF):
#         RF_START_PHASE= start_index- int(0.5*Fs) 
#         wave4[start_index- int(0.5*Fs) + i] = A_RF*np.cos( 2*np.pi*i*nu_RF/Fs) #generate RF signal to be sensed

#     for i in range(HALF_PI_E_LOCAL + 1): # pi/2 pulse
#         wave1[start_index + i] = A14*np.sin( 2*np.pi*i*nu14/Fs)+A23*np.sin( 2*np.pi*i*nu23/Fs)
#         wave2[start_index + i] = A14*np.cos( 2*np.pi*i*nu14/Fs) + A23*np.cos( 2*np.pi*i*nu23/Fs)

#     for j in range(N):
#         for i in range(ONE_PI_E_LOCAL+ 1):
#             wave1[start_index + HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length)] = A14*np.sin(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu14/Fs + np.pi/2) + A23*np.sin(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu23/Fs + np.pi/2)
#             wave2[start_index + HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length)] = A14*np.cos(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu14/Fs + np.pi/2) +A23*np.cos(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu23/Fs + np.pi/2)
        
#     for i in range(HALF_PI_E_LOCAL + 1):
#         wave1[start_index + HALF_PI_E_LOCAL+ N*(ONE_PI_E_LOCAL + 2*current_length) + i] = A14*np.sin(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu14/Fs) + A23*np.sin(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu23/Fs)
#         wave2[start_index + HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i] = A14*np.cos(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu14/Fs) +  A23*np.cos(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu23/Fs)
    
#     # Store to the nuclear spin
#     start_index = start_index + T_SENSING
#     for i in range(ONE_PI_E + 1):
#         wave1[start_index + i] = A14*np.sin(2*np.pi*i*nu14/Fs)
#         wave2[start_index + i] = A14*np.cos(2*np.pi*i*nu14/Fs)
    
#     for i in range(ONE_PI_N + 1):
#         wave3[start_index + i + ONE_PI_E+ T_MW_RF_separation] = A_n*np.sin(2*np.pi*i*RF_freq/Fs)    

#     # retrive 
#     start_index = start_index + T_SOTRE + T_INITIAL+ TAU_START + TAU_STEP*subcycle

#     for i in range(ONE_PI_N + 1):
#         wave3[start_index + i ] = A_n*np.sin(2*np.pi*i*RF_freq/Fs)  
    
#     for i in range(ONE_PI_E + 1):
#         wave1[start_index +ONE_PI_N + T_MW_RF_separation + i] = A14*np.sin(2*np.pi*i*nu14/Fs)
#         wave2[start_index +ONE_PI_N + T_MW_RF_separation + i] = A14*np.cos(2*np.pi*i*nu14/Fs)
    
#     # sensing with CPMG using local gate, after memory time
#     start_index = start_index +ONE_PI_N + T_MW_RF_separation + ONE_PI_E 
#     for i in range(T_RF):
#         PHASE_DIFF = start_index- int(0.5*Fs)-RF_START_PHASE
#         wave4[start_index- int(0.5*Fs) + i] = A_RF*np.cos( 2*np.pi*(i+PHASE_DIFF)*nu_RF/Fs) #generate RF signal to be sensed

#     for i in range(HALF_PI_E_LOCAL + 1): # pi/2 pulse
#         wave1[start_index + i] = A14*np.sin( 2*np.pi*i*nu14/Fs)+A23*np.sin( 2*np.pi*i*nu23/Fs)
#         wave2[start_index + i] = A14*np.cos( 2*np.pi*i*nu14/Fs) + A23*np.cos( 2*np.pi*i*nu23/Fs)

#     for j in range(N):
#         for i in range(ONE_PI_E_LOCAL+ 1):
#             wave1[start_index + HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length)] = A14*np.sin(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu14/Fs + np.pi/2) + A23*np.sin(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu23/Fs + np.pi/2)
#             wave2[start_index + HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length)] = A14*np.cos(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu14/Fs + np.pi/2) +A23*np.cos(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu23/Fs + np.pi/2)
        
#     for i in range(HALF_PI_E_LOCAL + 1):
#         wave1[start_index + HALF_PI_E_LOCAL+ N*(ONE_PI_E_LOCAL + 2*current_length) + i] = A14*np.sin(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu14/Fs) + A23*np.sin(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu23/Fs)
#         wave2[start_index + HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i] = A14*np.cos(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu14/Fs) +  A23*np.cos(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu23/Fs)
    
#     ## differetial measurement
#     start_index = subcycle * DATA_POINTS_PER_SUBCYCLE*2+ int(TAU_START*subcycle+TAU_STEP*subcycle*(subcycle-1)/2)*2+DATA_POINTS_PER_SUBCYCLE+ TAU_START+ TAU_STEP*subcycle + T_INITIAL + T_IDLE # starting position
#     # intialize nuclear spin
#     for i in range(ONE_PI_E + 1):
#         wave1[start_index + i] = A14*np.sin(2*np.pi*i*nu14/Fs) #nonlocal gate to flip the electron spin
#         wave2[start_index + i] = A14*np.cos(2*np.pi*i*nu14/Fs) #nonlocal gate to flip the electron spin
    
#     for i in range(ONE_PI_N + 1):
#         wave3[start_index + i + ONE_PI_E+ T_MW_RF_separation] = A_n*np.sin(2*np.pi*i*RF_freq/Fs)
    
#     # sensing with CPMG using local gate
#     start_index = start_index + T_WAITING + T_INITIAL
#     for i in range(T_RF):
#         RF_START_PHASE= start_index- int(0.5*Fs) 
#         wave4[start_index- int(0.5*Fs) + i] = A_RF*np.cos( 2*np.pi*i*nu_RF/Fs) #generate RF signal to be sensed

#     for i in range(HALF_PI_E_LOCAL + 1): # pi/2 pulse
#         wave1[start_index + i] = A14*np.sin( 2*np.pi*i*nu14/Fs)+A23*np.sin( 2*np.pi*i*nu23/Fs)
#         wave2[start_index + i] = A14*np.cos( 2*np.pi*i*nu14/Fs) + A23*np.cos( 2*np.pi*i*nu23/Fs)

#     for j in range(N):
#         for i in range(ONE_PI_E_LOCAL+ 1):
#             wave1[start_index + HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length)] = A14*np.sin(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu14/Fs + np.pi/2) + A23*np.sin(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu23/Fs + np.pi/2)
#             wave2[start_index + HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length)] = A14*np.cos(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu14/Fs + np.pi/2) +A23*np.cos(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu23/Fs + np.pi/2)
        
#     for i in range(HALF_PI_E_LOCAL + 1):
#         wave1[start_index + HALF_PI_E_LOCAL+ N*(ONE_PI_E_LOCAL + 2*current_length) + i] = -A14*np.sin(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu14/Fs) + A23*np.sin(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu23/Fs)
#         wave2[start_index + HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i] = -A14*np.cos(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu14/Fs) +  A23*np.cos(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu23/Fs)
    
#     # Store to the nuclear spin
#     start_index = start_index + T_SENSING
#     for i in range(ONE_PI_E + 1):
#         wave1[start_index + i] = A14*np.sin(2*np.pi*i*nu14/Fs)
#         wave2[start_index + i] = A14*np.cos(2*np.pi*i*nu14/Fs)
    
#     for i in range(ONE_PI_N + 1):
#         wave3[start_index + i + ONE_PI_E+ T_MW_RF_separation] = A_n*np.sin(2*np.pi*i*RF_freq/Fs)    

#     # retrive 
#     start_index = start_index + T_SOTRE + T_INITIAL+ TAU_START + TAU_STEP*subcycle

#     for i in range(ONE_PI_N + 1):
#         wave3[start_index + i ] = A_n*np.sin(2*np.pi*i*RF_freq/Fs)  
    
#     for i in range(ONE_PI_E + 1):
#         wave1[start_index +ONE_PI_N + T_MW_RF_separation + i] = A14*np.sin(2*np.pi*i*nu14/Fs)
#         wave2[start_index +ONE_PI_N + T_MW_RF_separation + i] = A14*np.cos(2*np.pi*i*nu14/Fs)
    
#     # sensing with CPMG using local gate, after memory time
#     start_index = start_index +ONE_PI_N + T_MW_RF_separation + ONE_PI_E 
#     for i in range(T_RF):
#         PHASE_DIFF = start_index- int(0.5*Fs)-RF_START_PHASE
#         wave4[start_index- int(0.5*Fs) + i] = A_RF*np.cos( 2*np.pi*(i+PHASE_DIFF)*nu_RF/Fs) #generate RF signal to be sensed

#     for i in range(HALF_PI_E_LOCAL + 1): # pi/2 pulse
#         wave1[start_index + i] = A14*np.sin( 2*np.pi*i*nu14/Fs)+A23*np.sin( 2*np.pi*i*nu23/Fs)
#         wave2[start_index + i] = A14*np.cos( 2*np.pi*i*nu14/Fs) + A23*np.cos( 2*np.pi*i*nu23/Fs)

#     for j in range(N):
#         for i in range(ONE_PI_E_LOCAL+ 1):
#             wave1[start_index + HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length)] = A14*np.sin(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu14/Fs + np.pi/2) + A23*np.sin(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu23/Fs + np.pi/2)
#             wave2[start_index + HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length)] = A14*np.cos(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu14/Fs + np.pi/2) +A23*np.cos(2*np.pi*(HALF_PI_E_LOCAL + current_length + i + j*(ONE_PI_E_LOCAL + 2*current_length))*nu23/Fs + np.pi/2)
        
#     for i in range(HALF_PI_E_LOCAL + 1):
#         wave1[start_index + HALF_PI_E_LOCAL+ N*(ONE_PI_E_LOCAL + 2*current_length) + i] = -A14*np.sin(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu14/Fs) + A23*np.sin(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu23/Fs)
#         wave2[start_index + HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i] = -A14*np.cos(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu14/Fs) +  A23*np.cos(2*np.pi*( HALF_PI_E_LOCAL + N*(ONE_PI_E_LOCAL + 2*current_length) + i)*nu23/Fs)
    




## Upload waveforms

waveforms = Waveforms()
waveforms2 = Waveforms()
waveforms[10] = (wave1,wave2,None) # I-component (output channel 1), Q-component (output channel 2), marker
waveforms2[11] = (wave3,wave4,None) # I-component (output channel 1), Q-component (output channel 2), marker
# awg.commandtable.upload_to_device(ct)
# awg2.commandtable.upload_to_device(ct2)
awg.write_to_waveform_memory(waveforms)
awg2.write_to_waveform_memory(waveforms2)
awg.enable_sequencer(single=True)
awg2.enable_sequencer(single=True)